In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw
from time import localtime, strftime

import lutorpy as lua
require("nn")
require("optim")

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 7)

# Number of samples in each class

TEST_SAMPLE_NUMBER = 51731
TRAIN_SAMPLE_NUMBER = 257187

In [2]:
ZERO_PADDING = 6

def get_filename(index):
    return "src/Csv/{}.txt".format(str(index).zfill(ZERO_PADDING))

def get_framename(index):
    return "src/Frame/{}.jpg".format(str(index).zfill(ZERO_PADDING))

NUMBER_OF_PEDESTRIANS = 12273

def download_pedestrian(index):
    assert(0 <= index < NUMBER_OF_PEDESTRIANS), "pedestrian number should be between 0 and {max}; given number: {id}".format(
        max=NUMBER_OF_PEDESTRIANS-1, id=index)
    filename = get_filename(index)
    data = pd.read_csv(filename, index_col=0)
    return data

In [3]:
def make_view(data, length, only_odd_rows=True):
    data = np.array(data).flatten()
    len1 = len(data)
    len2 = length
    return np.lib.stride_tricks.as_strided(data, shape=(len1 - len2 + 1, len2),
                                                    strides=(data.dtype.itemsize,) * 2)[::2 if only_odd_rows else 1, :]

In [4]:
# array1: [sX1; sY1; sX2; sY2; ...]
# array2: [eX1; eY1; eX2; eY2; ...]
# output: [dist((sX1, sY1), (eX1, eY1)), dist((sX1, sY1), (eX1, eY1)),
#          dist((sX2, sY2), (eX2, eY2)), dist((sX2, sY2), (eX2, eY2)), ...]

def distance_for_each_point(array1, array2):
    array_length = len(array1)
    len2 = len(array2)
    assert (array_length == len2), "Arrays' sizes have to be equal (array1: {}, array2: {})".format(array_length, len2)
    
    if array1.ndim == 1:
        distance = np.linalg.norm((array1 - array2).reshape((int(array_length / 2), 2)), axis=1)
        result = np.array([[d, d] for d in distance]).flatten()
    else:
        result = np.array([distance_for_each_point(array1[i], array2[i]) for i in range(array_length)])
    
    return result

# metrics between real results (in tests) and predicted
def distance(test_results, predicted_results):
    return distance_for_each_point(np.array(test_results), predicted_results).sum() / TEST_SAMPLE_NUMBER / 2

# score between real results (in tests) and predicted
def score(test_results, predicted_results, baseline_results):
    test_results = np.array(test_results)
    predicted_results = np.array(predicted_results)
    baseline_results = np.array(baseline_results)
    return (1 - ((test_results - predicted_results) ** 2).sum()/((test_results - baseline_results) ** 2).sum())

In [5]:
# data may be present as [n_features * n_samples] or [n_samples * n_features] 
# usually algorithms require second variant but I prefer first

def to_model(df):
    df = np.array(df).T
    return df.reshape((int(df.shape[0] / 2), int(df.shape[1] * 2)))

def to_model_with_features(df):
    df = np.array(df)
    movement_vector = np.diff(df,axis=0)
    speed = np.power(movement_vector, 2)
    speed[:,::2] = -speed[:,::2]
    speed = np.sqrt(np.diff(speed)[:,::2])
    double_speed = np.zeros_like(movement_vector, dtype=np.float64)
    double_speed[:,::2] = double_speed[:,1::2] = speed
    last_speed = speed[-1:]
    mean_speed = np.mean(speed, axis=0)
    cort_coord = movement_vector / double_speed
    angle = np.arctan2(cort_coord[:,::2], cort_coord[:,1::2]) / np.pi * 4
    
    return (np.concatenate((to_model(df), to_model(movement_vector), speed.T, last_speed.T, np.array([mean_speed]).T,
                            angle.T), axis=1)).astype(int)
def from_model(npa):
    return npa.reshape((int(npa.shape[0] * 2), int(npa.shape[1] / 2))).T

In [6]:
# first simply algorithm to get some start
# more about it you can find in 'baseline_distance_between_real_points_and_predicted.ipynb'
def baseline(test_data, start_point_index=0, number_of_points_to_return=5):
    error_template = "Start point index should be less than last point. Start point index: {st}, last point index: {end}"
    assert (start_point_index < len(test_data) - 1), error_template.format(st=start_point_index, end=len(test_data) - 1)
    
    start_point = np.array(test_data)[start_point_index]
    last_but_one_point = np.array(test_data)[-2]
    end_point = np.array(test_data)[-1]
    
    distance = distance_for_each_point(end_point, start_point)
    normalized_motion_vector = (end_point - start_point) / distance
    normalized_motion_vector[np.where(distance == 0)] = 0
    last_vector_length = distance_for_each_point(end_point, last_but_one_point)
    
    motion_vector = normalized_motion_vector * last_vector_length
    result = []
    for i in range(number_of_points_to_return):
        result.append(end_point + (i + 1) * motion_vector)
        
    return np.array(result)

In [7]:
def compare_results(function, test_results, parameter_name, list_of_values, **other_parameters):
    result = [] # we will keep results for each configuration here

    for i, value in enumerate(list_of_values):
        print("{cur}/{num}: {t}".format(cur=i+1, num=len(list_of_values), t=strftime("%Y-%m-%d %H:%M:%S", localtime())))
        other_parameters[parameter_name] = value        
        predicted_results = function(**other_parameters)
        np.save("src/Logs/{date}_predicted_coordinates_{function_name}_{parameter_name}_".format(
                date=strftime("%Y%m%d", localtime()), function_name=function.__name__, parameter_name=parameter_name)
                + str(value), predicted_results)
        result.append(score(test_results, predicted_results, baseline(other_parameters['test_data'], start_point_index=2)))
        
    print("done! {time}".format(time=strftime("%Y-%m-%d %H:%M:%S", localtime())))
    print("Results: {}".format(result))
        
    ind = list_of_values
    number = len(ind)
    width = 2 / number
    result_bar = plt.bar(range(number), result, width, color='g')

    plt.ylabel('Average difference')
    plt.xlabel(parameter_name)
    plt.title("Difference between real points and predicted by {parameter_name} in {function_name}".format(
        function_name=function.__name__, parameter_name=parameter_name))
    plt.xticks(np.array(range(number)) + width/2, ind)
    plt.savefig("src/Plots/{date}_{function_name}_score_by_{parameter_name}_with_features.png".format(
        date=strftime("%Y%m%d", localtime()), function_name=function.__name__, parameter_name=parameter_name))    

In [8]:
# reading data from .csv files

test_data = pd.read_csv('src/test_data_points.csv', index_col=0)
train_data = pd.read_csv('src/train_data_points.csv', index_col=0)

# split ten frames in input and output data (we want to predict output by input)

test_results = test_data[5:10]
test_data = test_data[:5]

train_results = train_data[5:10]
train_data = train_data[:5]

In [75]:
# define model (predictor): 
model = nn.Sequential()

ninputs = to_model_with_features(test_data).shape[1]
noutputs = 10

model._add(nn.Linear(ninputs, noutputs))

<Lua table at 0x401f72e0>

In [76]:
# define a loss function to be minimized (mean-square error between predictions and groundtruth labels)
criterion = nn.MSECriterion()

In [77]:
# x -- parameters (the vector of trainable weights -- all the weights of the linear matrix of our model, plus one bias)
# dl_dx -- gradients of our loss function
x, dl_dx = model._getParameters()

In [78]:
# define index generator for data

max_ind = to_model(train_data).shape[0]

def generator():
    i = 0
    while True:
        yield i % max_ind
        i += 1
        
ind = generator()

In [79]:
train_data_model = to_model_with_features(train_data)
train_results_model = to_model(train_results)

In [80]:
# computes the value of the loss function at a given point x, and the gradient of that function with respect to x
def feval(x_new):
    # set x to x_new, if differnt
    x._copy(x_new)
    
    # select a new training sample
    nidx = next(ind)
    
    target = torch.fromNumpyArray(train_results_model[nidx].astype(float))
    inputs = torch.fromNumpyArray(train_data_model[nidx].astype(float))
    
    # reset gradients (gradients are always accumulated, to accommodate batch methods)
    dl_dx._zero()
    
    # evaluate the loss function and its derivative wrt x, for that sample
    loss_x = criterion._forward(model._forward(inputs), target)
    model._backward(inputs, criterion._backward(model.output, target))

    # return loss(x) and dloss/dx
    return loss_x, dl_dx

In [81]:
# Given the function above, we can now easily train the model using SGD.
# For that, we need to define four key parameters:
# * learning rate: the size of the step taken at each stochasticestimate of the gradient
# * weight decay, to regularize the solution (L2 regularization)
# * momentum term, to average steps over time
# * learning rate decay, to let the algorithm converge more precisely

lua.execute(''' sgd_params = {
   learningRateDecay = 1e-3,
   learningRateDecay = 1e-4,
   weightDecay = 0,
   momentum = 0
} ''')

In [82]:
# We're now good to go... all we have left to do is run over the dataset
# for a certain number of iterations, and perform a stochastic update 
# at each iteration. The number of iterations is found empirically here,
# but should typically be determinined using cross-validation.

# we cycle over our training data
for i in range(100):

    # this variable is used to estimate the average loss
    current_loss = 0

    # an epoch is a full loop over our training data
    for i in range(max_ind):

        # optim contains several optimization algorithms. 
        # all of these algorithms assume the same parameters:
        # * closure that computes the loss, and its gradient wrt to x, given a point x
        # * point x
        # * some parameters, which are algorithm-specific
      
        _,fs = optim.sgd(feval,x,sgd_params)

        # functions in optim all return two things:
        # * the new x, found by the optimization method (here SGD)
        # * the value of the loss functions at all points that were used by the algorithm
        # (SGD only estimates the function once, so that list just contains one value)

        current_loss = current_loss + fs[1]

LuaError: /home/gaydashenko/torch/install/share/lua/5.1/optim/sgd.lua:82: invalid arguments: DoubleTensor number nil 
expected arguments: *DoubleTensor* [DoubleTensor] double | *DoubleTensor* [DoubleTensor] [double] DoubleTensor